###  ReAct Prompting
- Combines reasoning (Thought) with action (Act) and observation (Obs), allowing the model to interact with the environment.
- Uses external tools, APIs, or search engines to fetch real-time or specialized data.
- Combining COT & ReAct is often the best approach: Using ReAct to fetch data and CoT to reason through it. 

User query:

“I’m traveling from Bangalore to Delhi for 3 days. Find the cheapest flight, suggest a hotel within ₹4,000/night, and estimate total trip cost.”

In [11]:
from langchain_core.tools import tool 


@tool 
def search_flights(origin:str , destination:str , date:str) -> dict:
    """Search for the cheapest flights"""
    return {'airline':'Indigo' , 'price':'4500' , 'arrival':'4.00' , 'departure':'6.00'}

@tool
def search_hotels(city:str , max_price: int)-> dict:
    """Search for the best hotel under budget"""
    return {'hotel':'Novotel' , "price_per_night": 3500,"rating": 4.2}

@tool
def  calculate_total_cost(flight_price :int , hotel_price: int , night: int )-> int:
    """Returns the totall cost"""
    return flight_price +  (hotel_price * night)

@tool
def format_flight_summary(hotel :dict , flight : dict ,  total_cost: int)-> str:
    """Formats flight plan summary"""
    return (
        f"FLIGHT: {flight['airline']}, {flight['price']},{flight['departure']}, {flight['arrival']}\n"
        f"HOTEL :{hotel['hotel']}, {hotel['price_per_night']}/night , {hotel['rating']}\n"
        f"TOTAL COST: {total_cost}"
    )

In [12]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os 
from dotenv import load_dotenv
load_dotenv()

llm = ChatGroq(
    model="llama-3.1-8b-instant",   
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0
)

llm_with_tools = llm.bind_tools(
    [search_flights , search_hotels , calculate_total_cost , format_flight_summary] 
)

In [13]:
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an intelligent travel planning assistant that follows the ReAct framework "
     "(Reason → Act → Observe → Final Answer).\n\n"

     "Your responsibilities:\n"
     "1. Understand the user's travel intent, cities, dates, duration, and budget.\n"
     "2. Reason internally about the best steps to solve the task.\n"
     "3. Act by calling the appropriate tools when external data or calculations are needed.\n"
     "4. Observe tool results and use them to inform the next step.\n"
     "5. Repeat Reason → Act → Observe as needed.\n"
     "6. When all required information is gathered, produce a final, user-friendly answer and stop.\n\n"

     "Tool usage rules:\n"
     "- Always call tools for flight search, hotel search, and cost calculation.\n"
     "- Never fabricate prices, schedules, or availability.\n"
     "- If a tool fails or returns empty results, explain clearly and suggest alternatives.\n\n"

     "Response formatting guidelines:\n"
     "- Present results as a structured itinerary with labeled sections.\n"
     "- Include prices, timings, and key details.\n"
     "- End with a concise recommendation or summary.\n\n"

     "Behavioral constraints:\n"
     "- Be polite, professional, and concise.\n"
     "- Do not reveal internal reasoning steps.\n"
     "- Once the final answer is produced, stop responding.\n"
    ),
    ("user", "{question}")
])


### response.tool_calls :list of tool invocation requests generated by the LLM when it decides it needs  functions/tools to complete the task.

-  Each item represents one tool call with:
- Which tool to call
- What arguments to pass
- A unique call ID

In [14]:
def run_agent(question: str):
    messages = prompt.format_messages(question=question)
    
    while True:
        response = llm_with_tools.invoke(messages)
        
        # Add assistant message
        messages.append(response)

        if response.tool_calls:
            for tool_call in response.tool_calls:
                tool_name = tool_call["name"]
                args = tool_call["args"]
                
                tool_fn = {
                    "search_flights": search_flights,
                    "search_hotels": search_hotels,
                    "calculate_total_cost": calculate_total_cost,
                    "format_flight_summary": format_flight_summary
                }[tool_name]
                
                result = tool_fn.invoke(args)
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call["id"],
                    "content": str(result)
                })
        else:
            
            return response.content


In [15]:
result = run_agent("I’m traveling from Bangalore to Delhi for 3 days. Find the cheapest flight, suggest a hotel within ₹4,000/night, and estimate total trip cost.")
print(result)


**Trip Itinerary:**

**Flight:**

* From: Bangalore
* To: Delhi
* Airline: Indigo
* Price: ₹4,500
* Departure: 6:00 AM
* Arrival: 4:00 PM

**Hotel:**

* Name: Novotel
* Rating: 4.2/5
* Price per night: ₹3,500
* Location: Delhi

**Total Trip Cost:**

* Estimated total cost for 3 days: ₹38,000

**Recommendation:**

Book the Indigo flight from Bangalore to Delhi and stay at the Novotel hotel in Delhi. The estimated total cost for the 3-day trip is ₹38,000.
